In [ ]:
pip install sodapy
pip install boto3

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import pickle
import boto3

from datetime import date
from sodapy import Socrata

In [ ]:
# code for dataset of food inspections
chicago_dataset = "4ijn-s7e5"

### 1. Función get_client

In [ ]:
def get_client():
    client = Socrata("data.cityofchicago.org",
                 token="MyAppToken",
                 userame="user@example.com",
                 password="AFakePassword")
    
    return client

### 2. Ingesta inicial

Bajar los datos de la API de inspecciones de establecimientos de comida. 

In [ ]:
def ingesta_inicial(chicago_dataset, client, limit):
    datasets = client.get(chicago_dataset, limit=limit, offset=0, order='inspection_date')
    
    return datasets

In [ ]:
client = get_client()
datasets = ingesta_inicial(chicago_dataset, client, 300000)

TypeError: ignored

In [ ]:
len(datasets)

216490

In [ ]:
datasets[0]

{'inspection_id': '70269',
 'dba_name': "mr.daniel's",
 'aka_name': "mr.daniel's",
 'license_': '1899292',
 'facility_type': 'Restaurant',
 'risk': 'Risk 1 (High)',
 'address': '5645 W BELMONT AVE ',
 'city': 'CHICAGO',
 'state': 'IL',
 'zip': '60634',
 'inspection_date': '2010-01-04T00:00:00.000',
 'inspection_type': 'License Re-Inspection',
 'results': 'Pass',
 'latitude': '41.93844282365204',
 'longitude': '-87.76831838068422',
 'location': {'latitude': '-87.76831838068422',
  'longitude': '41.93844282365204'}}

### 3. Almacenamiento de ingesta inicial

In [ ]:
def guardar_ingesta(bucket, bucket_path, dataset):
    session = boto3.Session(
        aws_access_key_id = config['s3']['aws_access_key_id'],
        aws_secret_access_key = config['s3']['aws_secret_access_key']
    )

    s3 = session.resource('s3')

    s3.Object(bucket, bucket_path).put(Body=dataset)

In [ ]:
TODAY = date.today()

In [ ]:
pickle_data = pickle.dumps(datasets)

In [ ]:
bucket = "aplicaciones-cd-1"
key = "ingesta/inicial/inspecciones-historicas-" + str(TODAY) + ".pkl"

guardar_ingesta(bucket, key, pickle_data)

Guardar el dataset en un pickle en un bucket de s3, el nombre del archivo dete terminar con el día en el que se realizó la ingesta (obtenido de manera dinámica). 

### 4. Ingestas consecutivas

La siguiente vez que se ingestan los datos se tendrá que pedir específicamente a partir de la fecha desde la que no tienes datos.

In [ ]:
def ingesta_consecutiva(chicago_dataset, client, fecha, limit):
    new_dataset = client.get(chicago_dataset, limit=limit, where="inspection_date>='{}'".format(fecha))
    
    return new_dataset

In [ ]:
client = get_client()

In [ ]:
new_dataset = ingesta_consecutiva(chicago_dataset, client, '2020-11-03', 1000)

In [ ]:
new_dataset[0]

{'inspection_id': '2454202',
 'dba_name': 'TAAMI',
 'aka_name': 'TAAMI',
 'license_': '1921638',
 'risk': 'Risk 3 (Low)',
 'address': '2931 W TOUHY AVE ',
 'city': 'CHICAGO',
 'state': 'IL',
 'zip': '60645',
 'inspection_date': '2020-11-03T00:00:00.000',
 'inspection_type': 'Canvass',
 'results': 'Out of Business',
 'latitude': '42.01187856476175',
 'longitude': '-87.70337002780445',
 'location': {'latitude': '-87.70337002780445',
  'longitude': '42.01187856476175'}}

In [ ]:
len(new_dataset)

1000

### 5. Almacenamiento de ingestas consecutivas

In [ ]:
pickled_new_data = pickle.dumps(new_dataset)

In [ ]:
bucket = "aplicaciones-cd-1"
key = "ingesta/consecutiva/inspecciones-consecutivas-" + str(TODAY) + ".pkl"

guardar_ingesta(bucket, key, pickled_new_data)